In [1]:
pip install spotipy 

     |████████████████████████████████| 61kB 3.7MB/s 
     |████████████████████████████████| 153kB 24.3MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="a006ea8174bc4689b4eb39c47b5449a1" 
secret = "1cca3d1fff6145fdaee72ba822e8b586" 

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
import timeit
import pandas as pd
import torch

In [4]:
def getTracks(query, number):
  start = timeit.default_timer()

  # create empty lists where the results are going to be stored
  artist_name = []
  track_name = []
  popularity = []
  track_id = []

  for i in range(0,number,10):
      track_results = sp.search(q=query, type='track', limit=10,offset=i)
      for i, t in enumerate(track_results['tracks']['items']):
          artist_name.append(t['artists'][0]['name'])
          track_name.append(t['name'])
          track_id.append(t['id'])
          popularity.append(t['popularity'])
        

  stop = timeit.default_timer()
  print ('Time to run this code (in seconds):', stop - start)
  df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
  return df_tracks

In [57]:
df_calm_tracks = getTracks('label:calm',300)

Time to run this code (in seconds): 1.3527960510000412


In [58]:
print(df_calm_tracks.shape)
df_calm_tracks.head()

(300, 4)


,artist_name,track_name,track_id,popularity
0,The Calm Factory,Forest Thunderstorm,3NTAcEKTrL3yHnEzBklXcN,63
1,The Calm Factory,Calm Before The Storm,69mYHeNk3DSNIRBUCLCFZI,60
2,The Calm Factory,Rain For Relaxation,5WAqDgplhuoFBG4IUdEP4J,59
3,The Calm Factory,Chuva soa para dormir,5sMzZKg6SLtG5lcudEpDLC,58
4,Sounds Of Calm,Small River On Rocks,1iwAjsOxILo4p6uOZLD0Fa,47


In [59]:
df_energetic_tracks = getTracks('label:energetic',300)

Time to run this code (in seconds): 1.367311993000044


In [60]:
print(df_energetic_tracks.shape)
df_energetic_tracks.head()

(300, 4)


,artist_name,track_name,track_id,popularity
0,Lil Key,Crazy (feat. J.I. & D Low),5XgfOmcWvkXTDmOfO81NRP,14
1,Energetic Alchemy,The Lovers,6cVj2hWi6hDovzI6fjWH3N,9
2,Sharon Hart,Near Reject Pick,3d0t8krXtXnNsCPXruNWqR,9
3,Kasperli,De Seegeischt im Führwehrweiher - Teil 10,1dkMVDHOBz2r171zApCVdr,10
4,Kasperli,De Seegeischt im Führwehrweiher - Teil 9,4vlFuPLQFmhMsw2u75PXuJ,10


In [61]:
df_happy_tracks = getTracks('label:happy',300)

Time to run this code (in seconds): 1.2082732349999787


In [62]:
print(df_happy_tracks.shape)
df_happy_tracks.head()

(300, 4)


,artist_name,track_name,track_id,popularity
0,L.B. One,My Mother Told Me - Vikings Anthem,6pAzrdqZKo4AnRWDKStG7R,65
1,The Happy Fits,Too Late,3LbO439QNr3IX7Ho39Fq8X,59
2,Lil Happy Lil Sad,Let Me Die,46T7puWlH8ptC6lx7bQ3TY,66
3,The Happy Fits,Dirty Imbecile,5erGUceK8ieHWtTXdnGP65,57
4,Sad Frosty,Beavis & Butt-head,0Y9kRbsQKC9zSmfzxPgYiT,56


In [63]:
df_sad_tracks = getTracks('label:sad',300)

Time to run this code (in seconds): 1.369971185000054


In [64]:
print(df_sad_tracks.shape)
df_sad_tracks.head()

(300, 4)


,artist_name,track_name,track_id,popularity
0,Lil Happy Lil Sad,Let Me Die,46T7puWlH8ptC6lx7bQ3TY,66
1,Sad Monologue,Cabin Jam,3McFNMvwAOUOCM83wyTpUr,59
2,The Licks,Lavender Kiss,70oY6gKgHfMlVLVxFROeiA,57
3,Ramiro sad,Abismo,0WeBxkwIGnnoPAPNsnfEAd,50
4,Sad Ul,Wasteland Worry,0YsE50FuGtzivL2t8CeG9h,44


In [13]:
def cleanDuplicates(tracks):
  grouped = tracks.groupby(['artist_name','track_name'], as_index=True).size()
  grouped[grouped > 1].count()
  tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)
  

In [65]:
cleanDuplicates(df_calm_tracks)

In [66]:
grouped_after_dropping = df_calm_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [67]:
cleanDuplicates(df_energetic_tracks)
cleanDuplicates(df_happy_tracks)
cleanDuplicates(df_sad_tracks)

In [68]:
grouped_after_dropping = df_energetic_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [69]:
grouped_after_dropping = df_sad_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [19]:
def getAudioFeatures(tracks):
  # again measuring the time
  start = timeit.default_timer()

  # empty list, batchsize and the counter for None results
  rows = []
  batchsize = 100
  None_counter = 0

  for i in range(0,len(tracks['track_id']),batchsize):
      batch = tracks['track_id'][i:i+batchsize]
      feature_results = sp.audio_features(batch)
      for i, t in enumerate(feature_results):
          if t == None:
              None_counter = None_counter + 1
          else:
              rows.append(t)
              
  print('Number of tracks where no audio features were available:',None_counter)

  stop = timeit.default_timer()
  print ('Time to run this code (in seconds):',stop - start)
  df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
  return df_audio_features

In [70]:
df_calm_audio_features = getAudioFeatures(df_calm_tracks)

Number of tracks where no audio features were available: 8
Time to run this code (in seconds): 0.5055865629999516


In [71]:
print("Shape of the dataset:", df_calm_audio_features.shape)
df_calm_audio_features.head()

Shape of the dataset: (286, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.142,0.516,1,-25.856,1,0.0694,0.730000,0.944,0.223,0.0263,88.277,audio_features,3NTAcEKTrL3yHnEzBklXcN,spotify:track:3NTAcEKTrL3yHnEzBklXcN,https://api.spotify.com/v1/tracks/3NTAcEKTrL3y...,https://api.spotify.com/v1/audio-analysis/3NTA...,179972,4
1,0.142,0.320,1,-32.837,1,0.0691,0.782000,0.963,0.103,0.0373,139.713,audio_features,69mYHeNk3DSNIRBUCLCFZI,spotify:track:69mYHeNk3DSNIRBUCLCFZI,https://api.spotify.com/v1/tracks/69mYHeNk3DSN...,https://api.spotify.com/v1/audio-analysis/69mY...,241556,4
2,0.194,0.952,7,-17.896,1,0.0888,0.002150,0.996,0.687,0.0309,153.467,audio_features,5WAqDgplhuoFBG4IUdEP4J,spotify:track:5WAqDgplhuoFBG4IUdEP4J,https://api.spotify.com/v1/tracks/5WAqDgplhuoF...,https://api.spotify.com/v1/audio-analysis/5WAq...,157304,3
3,0.248,0.963,9,-18.139,1,0.0865,0.000776,0.987,0.365,0.0289,76.466,audio_features,5sMzZKg6SLtG5lcudEpDLC,spotify:track:5sMzZKg6SLtG5lcudEpDLC,https://api.spotify.com/v1/tracks/5sMzZKg6SLtG...,https://api.spotify.com/v1/audio-analysis/5sMz...,140142,3
4,0.288,0.949,0,-27.451,1,0.1100,0.211000,0.929,0.882,0.0227,89.076,audio_features,1iwAjsOxILo4p6uOZLD0Fa,spotify:track:1iwAjsOxILo4p6uOZLD0Fa,https://api.spotify.com/v1/tracks/1iwAjsOxILo4...,https://api.spotify.com/v1/audio-analysis/1iwA...,243533,4


In [72]:
df_energetic_audio_features = getAudioFeatures(df_energetic_tracks)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 0.4839851810000937


In [73]:
print("Shape of the dataset:", df_energetic_audio_features.shape)
df_energetic_audio_features.head()

Shape of the dataset: (289, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.790,0.703,7,-4.285,1,0.1150,0.20700,0.000000,0.1140,0.399,139.995,audio_features,5XgfOmcWvkXTDmOfO81NRP,spotify:track:5XgfOmcWvkXTDmOfO81NRP,https://api.spotify.com/v1/tracks/5XgfOmcWvkXT...,https://api.spotify.com/v1/audio-analysis/5Xgf...,292781,4
1,0.466,0.680,11,-6.729,0,0.0408,0.01540,0.000047,0.1500,0.221,99.943,audio_features,6cVj2hWi6hDovzI6fjWH3N,spotify:track:6cVj2hWi6hDovzI6fjWH3N,https://api.spotify.com/v1/tracks/6cVj2hWi6hDo...,https://api.spotify.com/v1/audio-analysis/6cVj...,216059,4
2,0.811,0.901,11,-5.390,0,0.0389,0.00211,0.848000,0.0425,0.725,127.994,audio_features,3d0t8krXtXnNsCPXruNWqR,spotify:track:3d0t8krXtXnNsCPXruNWqR,https://api.spotify.com/v1/tracks/3d0t8krXtXnN...,https://api.spotify.com/v1/audio-analysis/3d0t...,131250,4
3,0.737,0.434,6,-14.013,1,0.9020,0.54000,0.000000,0.3320,0.634,101.835,audio_features,1dkMVDHOBz2r171zApCVdr,spotify:track:1dkMVDHOBz2r171zApCVdr,https://api.spotify.com/v1/tracks/1dkMVDHOBz2r...,https://api.spotify.com/v1/audio-analysis/1dkM...,115693,4
4,0.626,0.453,8,-13.727,0,0.9210,0.61900,0.000000,0.3650,0.687,76.235,audio_features,4vlFuPLQFmhMsw2u75PXuJ,spotify:track:4vlFuPLQFmhMsw2u75PXuJ,https://api.spotify.com/v1/tracks/4vlFuPLQFmhM...,https://api.spotify.com/v1/audio-analysis/4vlF...,84880,3


In [74]:
df_happy_audio_features = getAudioFeatures(df_happy_tracks)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 0.48248773000000256


In [75]:
print("Shape of the dataset:", df_happy_audio_features.shape)
df_happy_audio_features.head()

Shape of the dataset: (298, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.650,0.971,5,-4.088,0,0.2090,0.00656,0.0327,0.2580,0.200,139.970,audio_features,6pAzrdqZKo4AnRWDKStG7R,spotify:track:6pAzrdqZKo4AnRWDKStG7R,https://api.spotify.com/v1/tracks/6pAzrdqZKo4A...,https://api.spotify.com/v1/audio-analysis/6pAz...,197250,4
1,0.685,0.554,7,-10.473,1,0.1150,0.44300,0.0000,0.4820,0.789,160.002,audio_features,3LbO439QNr3IX7Ho39Fq8X,spotify:track:3LbO439QNr3IX7Ho39Fq8X,https://api.spotify.com/v1/tracks/3LbO439QNr3I...,https://api.spotify.com/v1/audio-analysis/3LbO...,201258,4
2,0.503,0.370,11,-17.502,1,0.0489,0.76800,0.5990,0.1160,0.102,78.963,audio_features,46T7puWlH8ptC6lx7bQ3TY,spotify:track:46T7puWlH8ptC6lx7bQ3TY,https://api.spotify.com/v1/tracks/46T7puWlH8pt...,https://api.spotify.com/v1/audio-analysis/46T7...,218776,4
3,0.834,0.531,3,-10.450,1,0.1600,0.48000,0.0000,0.0957,0.390,99.999,audio_features,5erGUceK8ieHWtTXdnGP65,spotify:track:5erGUceK8ieHWtTXdnGP65,https://api.spotify.com/v1/tracks/5erGUceK8ieH...,https://api.spotify.com/v1/audio-analysis/5erG...,239612,4
4,0.848,0.475,0,-8.949,1,0.4760,0.01380,0.0000,0.1070,0.195,138.917,audio_features,0Y9kRbsQKC9zSmfzxPgYiT,spotify:track:0Y9kRbsQKC9zSmfzxPgYiT,https://api.spotify.com/v1/tracks/0Y9kRbsQKC9z...,https://api.spotify.com/v1/audio-analysis/0Y9k...,124317,4


In [76]:
df_sad_audio_features = getAudioFeatures(df_sad_tracks)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 0.4957657759999847


In [77]:
print("Shape of the dataset:", df_sad_audio_features.shape)
df_sad_audio_features.head()

Shape of the dataset: (297, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.503,0.370,11,-17.502,1,0.0489,0.76800,0.5990,0.1160,0.102,78.963,audio_features,46T7puWlH8ptC6lx7bQ3TY,spotify:track:46T7puWlH8ptC6lx7bQ3TY,https://api.spotify.com/v1/tracks/46T7puWlH8pt...,https://api.spotify.com/v1/audio-analysis/46T7...,218776,4
1,0.477,0.370,4,-12.394,1,0.0315,0.68900,0.7990,0.0977,0.638,179.707,audio_features,3McFNMvwAOUOCM83wyTpUr,spotify:track:3McFNMvwAOUOCM83wyTpUr,https://api.spotify.com/v1/tracks/3McFNMvwAOUO...,https://api.spotify.com/v1/audio-analysis/3McF...,192710,4
2,0.578,0.115,0,-20.333,1,0.0360,0.00922,0.0031,0.0834,0.363,199.975,audio_features,70oY6gKgHfMlVLVxFROeiA,spotify:track:70oY6gKgHfMlVLVxFROeiA,https://api.spotify.com/v1/tracks/70oY6gKgHfMl...,https://api.spotify.com/v1/audio-analysis/70oY...,243646,3
3,0.677,0.416,0,-11.775,1,0.0313,0.42100,0.0000,0.1350,0.172,126.174,audio_features,0WeBxkwIGnnoPAPNsnfEAd,spotify:track:0WeBxkwIGnnoPAPNsnfEAd,https://api.spotify.com/v1/tracks/0WeBxkwIGnno...,https://api.spotify.com/v1/audio-analysis/0WeB...,174762,4
4,0.543,0.788,5,-0.949,0,0.3180,0.12900,0.0000,0.4210,0.468,142.784,audio_features,0YsE50FuGtzivL2t8CeG9h,spotify:track:0YsE50FuGtzivL2t8CeG9h,https://api.spotify.com/v1/tracks/0YsE50FuGtzi...,https://api.spotify.com/v1/audio-analysis/0YsE...,64016,4


In [28]:
def dropColumns(df_audio_features):
  columns_to_drop = ['analysis_url','track_href','type','key','mode','time_signature']
  df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

  df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

  df_audio_features.shape

In [78]:
dropColumns(df_calm_audio_features)
dropColumns(df_energetic_audio_features)
dropColumns(df_happy_audio_features)
dropColumns(df_sad_audio_features)

In [79]:
if not 'mood' in df_calm_audio_features:
  df_calm_audio_features['mood'] = 'calm' # label calm 

if not 'mood' in df_energetic_audio_features:
  df_energetic_audio_features['mood'] = 'energetic' # label energetic 

if not 'mood' in df_happy_audio_features:
  df_happy_audio_features['mood'] = 'happy' # label happy

if not 'mood' in df_sad_audio_features:
  df_sad_audio_features['mood'] = 'sad' # label sad 

In [80]:
def mergeDataframes(df_tracks,df_audio_features):
  # merge both dataframes
  # the 'inner' method will make sure that we only keep track IDs present in both datasets
  df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
  
  return df

In [81]:
df_calm = mergeDataframes(df_calm_tracks, df_calm_audio_features)
df_energetic = mergeDataframes(df_energetic_tracks, df_energetic_audio_features)
df_happy = mergeDataframes(df_happy_tracks, df_happy_audio_features)
df_sad = mergeDataframes(df_sad_tracks, df_sad_audio_features)

In [82]:
df_calm.to_csv('calm.csv')
df_energetic.to_csv('energetic.csv')
df_happy.to_csv('happy.csv')
df_sad.to_csv('sad.csv')

In [34]:
%mkdir data_from_spotify

In [83]:
!mv "/content/calm.csv" "/content/data_from_spotify"

In [84]:
!mv "/content/energetic.csv" "/content/data_from_spotify"

In [85]:
!mv "/content/happy.csv" "/content/data_from_spotify"

In [86]:
!mv "/content/sad.csv" "/content/data_from_spotify"

In [87]:
df_list = [df_calm, df_energetic, df_happy, df_sad]

In [88]:
df = pd.concat(df_list)

In [89]:
df

,artist_name,track_name,track_id,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,mood
0,The Calm Factory,Forest Thunderstorm,3NTAcEKTrL3yHnEzBklXcN,63,0.142,0.516,-25.856,0.0694,0.730000,0.944000,0.223,0.0263,88.277,spotify:track:3NTAcEKTrL3yHnEzBklXcN,179972,calm
1,The Calm Factory,Calm Before The Storm,69mYHeNk3DSNIRBUCLCFZI,60,0.142,0.320,-32.837,0.0691,0.782000,0.963000,0.103,0.0373,139.713,spotify:track:69mYHeNk3DSNIRBUCLCFZI,241556,calm
2,The Calm Factory,Rain For Relaxation,5WAqDgplhuoFBG4IUdEP4J,59,0.194,0.952,-17.896,0.0888,0.002150,0.996000,0.687,0.0309,153.467,spotify:track:5WAqDgplhuoFBG4IUdEP4J,157304,calm
3,The Calm Factory,Chuva soa para dormir,5sMzZKg6SLtG5lcudEpDLC,58,0.248,0.963,-18.139,0.0865,0.000776,0.987000,0.365,0.0289,76.466,spotify:track:5sMzZKg6SLtG5lcudEpDLC,140142,calm
4,Sounds Of Calm,Small River On Rocks,1iwAjsOxILo4p6uOZLD0Fa,47,0.288,0.949,-27.451,0.1100,0.211000,0.929000,0.882,0.0227,89.076,spotify:track:1iwAjsOxILo4p6uOZLD0Fa,243533,calm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,Chris Pureka,Back in the Ring,6UXpeNbXigZehwcaediKiV,15,0.331,0.532,-5.826,0.0290,0.392000,0.000148,0.110,0.1390,98.194,spotify:track:6UXpeNbXigZehwcaediKiV,306653,sad
293,Sad Penny,Pissing,2a0x3B5aG0AAH7gyQRX6lC,20,0.495,0.518,-6.055,0.0361,0.001790,0.058500,0.119,0.1750,117.999,spotify:track:2a0x3B5aG0AAH7gyQRX6lC,248208,sad
294,KOTAROKILLA,Знаки,5gEbC1FTcZ3NOe81S6GfH4,15,0.717,0.519,-14.988,0.1400,0.496000,0.001250,0.106,0.2080,120.091,spotify:track:5gEbC1FTcZ3NOe81S6GfH4,136460,sad
295,Sad Cops,Best Friends,3npR9KfMOzwOWXvFkdnoDT,16,0.662,0.612,-11.619,0.0398,0.037100,0.722000,0.160,0.8110,92.993,spotify:track:3npR9KfMOzwOWXvFkdnoDT,171200,sad


In [90]:
df = df.sample(frac=1).reset_index(drop=True)

In [91]:
df

,artist_name,track_name,track_id,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,mood
0,Sad Puppy,Thinking of You,1rkIK8cog9QgBYmpT2XOdv,46,0.848,0.522,-4.180,0.0411,0.5000,0.000288,0.0786,0.797,94.970,spotify:track:1rkIK8cog9QgBYmpT2XOdv,154163,sad
1,Oscar Louis,The Rest,2SRx7uOuHLWrUuN2dyrZ41,31,0.697,0.400,-11.350,0.0316,0.5850,0.024200,0.1100,0.441,158.014,spotify:track:2SRx7uOuHLWrUuN2dyrZ41,140209,happy
2,Happy Moon,When I Met You,4xZ4ucmEqNuGLKsVJUo17T,32,0.551,0.410,-8.691,0.0378,0.1760,0.752000,0.0944,0.158,119.969,spotify:track:4xZ4ucmEqNuGLKsVJUo17T,87159,happy
3,Calm Baby,"Row, Row, Row Your Boat",2quUTb4VsSMVCRQoQ2ITbb,15,0.164,0.208,-12.219,0.0368,0.8530,0.000603,0.1060,0.256,176.301,spotify:track:2quUTb4VsSMVCRQoQ2ITbb,68280,calm
4,Kasperli,De Pischima ufem Burehof - Teil 11,52mkFTivDBkwQ29aTKDvuI,16,0.701,0.456,-13.095,0.9530,0.6430,0.000000,0.3160,0.453,96.649,spotify:track:52mkFTivDBkwQ29aTKDvuI,106839,energetic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1165,Simpson,I Miss My Dawgs,1nkWeF77qk4RkLpn4omPWL,50,0.779,0.309,-11.634,0.0662,0.0951,0.004500,0.3300,0.740,75.007,spotify:track:1nkWeF77qk4RkLpn4omPWL,142472,happy
1166,Fucking Beautiful,Wet Dream Baby,4XW3aBxh0Gf0KE49hNc733,1,0.326,0.950,-3.465,0.2290,0.0725,0.000003,0.0797,0.719,169.762,spotify:track:4XW3aBxh0Gf0KE49hNc733,156000,energetic
1167,Kasperli,De Seegeischt im Führwehrweiher - Teil 2,4AnYrKR010Rdc8oqloVdHK,11,0.746,0.253,-16.731,0.9440,0.6950,0.000000,0.3860,0.786,95.313,spotify:track:4AnYrKR010Rdc8oqloVdHK,94680,energetic
1168,Badanamu,Po Pow Pay,3KdcrLnjSfnE0zYOe0ekRJ,35,0.974,0.769,-7.622,0.0583,0.0631,0.844000,0.2760,0.782,128.004,spotify:track:3KdcrLnjSfnE0zYOe0ekRJ,121875,calm


In [92]:
#Libraries to pre-process the variables
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split

In [114]:
col_features = df.columns[4:13]
features = df[col_features]
mood = df['mood']
features = MinMaxScaler().fit_transform(features)
#Encode the labels (targets)
encoder = LabelEncoder()
encoder.fit(mood)
encoded_y = encoder.transform(mood)
#Split train and test data with a test size of 20%
features_train,features_test,mood_train,mood_test = train_test_split(features,encoded_y,test_size=0.2,random_state=15)

In [115]:
features.shape


(1170, 9)

In [116]:
len(features_train)

936

In [117]:
len(features_test)

234

In [118]:
len(mood_train)

936

In [119]:
len(mood_test)

234